# Test part of data pipeline where we merge all the hourly data

Breaks on full run: need to convert to dask df

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [72]:
# Start client so can see worker mem use
from dask.distributed import Client 
client = Client(n_workers=2, threads_per_worker=2, memory_limit="1GB") # limit worker size to create issues even on --small dataset
client

/Users/gailin.pease/miniconda3/envs/hourly_egrid/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52231 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52231/status,
Dashboard: http://127.0.0.1:52231/status,Workers: 2
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52235,Workers: 2
Dashboard: http://127.0.0.1:52231/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:52294,Total threads: 2
Dashboard: http://127.0.0.1:52296/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:52241,


In [64]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../hourly-egrid/")

# import local modules
import src.data_cleaning as data_cleaning

In [67]:
o_shaped_eia_data = pd.read_csv("../data/outputs/small/shaped_eia923_data_2020.csv", parse_dates=["datetime_utc"])
o_partial_cems_scaled = pd.read_csv("../data/outputs/small/partial_cems_scaled_2020.csv", parse_dates=["datetime_utc"]) #NOT FINAL VERSION 
o_cems = pd.read_csv("../data/outputs/small/cems_2020.csv", parse_dates=["datetime_utc"]) # NOT FINAL VERSION

# shaped_eia_data = pd.read_csv("../data/outputs/shaped_eia923_data_2020.csv")
# partial_cems_scaled = pd.read_csv("../data/outputs/partial_cems_scaled_2020.csv") #NOT FINAL VERSION 
# cems = pd.read_csv("../data/outputs/cems_2020.csv") # NOT FINAL VERSION

/var/folders/mr/3dcnnc3153j6s05bmdwr0r0h0000gq/T/ipykernel_66126/3263455066.py:3: DtypeWarning: Columns (1,13) have mixed types. Specify dtype option on import or set low_memory=False.
  o_cems = pd.read_csv("../data/outputs/small/cems_2020.csv", parse_dates=["datetime_utc"]) # NOT FINAL VERSION


In [68]:
# most of the stuff done to partial_cems_scaled and cems in data_pipeline is in plant_static_attributes: 
#plant_static_attributes = pd.read_csv("../data/results/plant_data/plant_static_attributes.csv")

plant_static_attributes = pd.read_csv("../data/results/small/plant_data/plant_static_attributes.csv")

In [69]:
o_partial_cems_scaled = o_partial_cems_scaled.merge(plant_static_attributes, how='left', on='plant_id_eia')
o_cems = o_cems.merge(plant_static_attributes, how='left', on='plant_id_eia')

In [70]:
# Problem data_pipeline steps: on small data, these are fine with worker size 16GB (break with worker size 1GB)
# On full dataset, these break with worker size 16GB (ie Gailin's laptop)

combined_plant_data = data_cleaning.combine_subplant_data(
        o_cems, o_partial_cems_scaled, o_shaped_eia_data
)

# 12. Aggregate CEMS data to BA-fuel and write power sector results
ba_fuel_data = data_cleaning.aggregate_plant_data_to_ba_fuel(
    combined_plant_data, plant_static_attributes
)

Plant data to dask
Merging plant data
Grouping plant data
Computing before returning


In [ ]:

#plants = combined_plant_data.groupby("plant_id_eia").sum().compute()

# What is the minimum dataset for which this breaks? 

In [56]:
o_shaped_eia_data.datetime_utc = pd.to_datetime(o_shaped_eia_data.datetime_utc)

In [58]:
#dt_shaped["datetime_utc"] = dt_shaped.datetime_utc.dt.to_datetime64()


In [78]:
dt_shaped = o_shaped_eia_data.set_index("datetime_utc")
shaped_eia_dask = dd.from_pandas(dt_shaped, npartitions=50)

In [79]:
shaped_eia_dask

,plant_id_eia,subplant_id,datetime_local,report_date,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_lb,ch4_mass_lb,n2o_mass_lb,nox_mass_lb,so2_mass_lb,co2_mass_lb_for_electricity,ch4_mass_lb_for_electricity,n2o_mass_lb_for_electricity,nox_mass_lb_for_electricity,so2_mass_lb_for_electricity,co2_mass_lb_adjusted,ch4_mass_lb_adjusted,n2o_mass_lb_adjusted,nox_mass_lb_adjusted,so2_mass_lb_adjusted,profile_method,hourly_data_source
npartitions=50,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 05:00:00+00:00,int64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object
2020-01-08 17:00:00+00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25 04:00:00+00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaT,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [80]:
smol = shaped_eia_dask[["net_generation_mwh","fuel_consumed_mmbtu"]]

In [81]:
# even this breaks. If can get this to work, maybe that solution works for rest? 
smol.groupby("datetime_utc").sum().compute()

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 672.44 MiB -- Worker memory limit: 0.93 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 671.75 MiB -- Worker memory limit: 0.93 GiB
distributed.worker - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 781.67 MiB -- Worker memory limit: 0.93 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 781.67 MiB -- Worker memory limit: 0.93 G

KilledWorker: ("('from_pandas-0a0fcc79d26224c4f0b45f484386384f', 49)", <WorkerState 'tcp://127.0.0.1:61704', name: 0, status: closed, memory: 0, processing: 28>)

distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 791.55 MiB -- Worker memory limit: 0.93 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 791.55 MiB -- Worker memory limit: 0.93 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker.html#memtrim for more information. -- Unmanaged memory: 791.59 MiB -- Worker memory limit: 0.93 GiB
distributed.worker - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS